In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel

sc.install_pypi_package('pandas')
sc.install_pypi_package('pyarrow==0.10.0')
sc.install_pypi_package('sklearn')

import pandas as pd
from sklearn import metrics



In [ ]:
spark=SparkSession.builder.appName('classification').getOrCreate()
pandas_df = pd.read_csv('affairs.csv')
sqlContest = SQLContext(sc)  
df = sqlContest.createDataFrame(pandas_df) 



print((df.count(),len(df.columns)))


df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'
                                         ,'gender','education','occupation'], 
                               outputCol="features")
df = df_assembler.transform(df)
df.printSchema()
df.show(5,truncate=False)


model_df=df.select(['features','affairs'])
train_df,test_df=model_df.randomSplit([0.75,0.25])
train_df.count()
train_df.groupBy('affairs').count().show()
test_df.groupBy('affairs').count().show()


rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)
rf_predictions=rf_classifier.transform(test_df)
rf_predictions.show()

rf_classifier.featureImportances 



rf_predictions.groupBy('prediction').count().show()
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)
print(rf_auc)


